In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

/home/uttaran/anaconda3/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Data has apparently already been downloaded and unpacked.


In [2]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')
tf.logging.set_verbosity(tf.logging.ERROR)

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [3]:
weight_decay = 1e-3
regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
filter_size = 3
feature_maps = 96

In [7]:
def cnn_model_fn(features, labels, mode):
      input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
      dropout0 = tf.contrib.layers.dropout(inputs=input_layer, keep_prob=0.8)  
      conv1 = tf.contrib.layers.conv2d(
          inputs = dropout0,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )
      
      conv1_bn =  tf.contrib.layers.batch_norm(
            inputs=conv1,
            fused = True
      )
        
      conv2 = tf.contrib.layers.conv2d(
          inputs = conv1_bn,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv2_bn =  tf.contrib.layers.batch_norm(
            inputs=conv2,
            fused = True
        )
      

      conv3 = tf.contrib.layers.conv2d(
          inputs = conv2_bn,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          stride = 2
          )
      conv3_bn =  tf.contrib.layers.batch_norm(
            inputs=conv3,
            fused = True
        )
      dropout1 = tf.contrib.layers.dropout(inputs=conv3_bn, keep_prob=0.5)
      conv4 = tf.contrib.layers.conv2d(
          inputs = dropout1,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv4_bn =  tf.contrib.layers.batch_norm(
            inputs=conv4,
            fused = True
        )

      
      #pool2 = tf.contrib.layers.max_pool2d(inputs = conv4_bn,kernel_size = [2,2],stride = 2)
    
      #dropout2 =  tf.contrib.layers.dropout(inputs=pool2, keep_prob=0.7)

      conv5 = tf.contrib.layers.conv2d(
          inputs = conv4_bn,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )


      conv5_bn =  tf.contrib.layers.batch_norm(
            inputs=conv5,
            fused = True
        )

      conv6 = tf.contrib.layers.conv2d(
          inputs = conv5_bn,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          stride = 2
          )

      conv6_bn =  tf.contrib.layers.batch_norm(
            inputs=conv6,
            fused = True
        )

      
      #pool3 = tf.contrib.layers.max_pool2d(inputs = conv6_bn,kernel_size = [2,2] ,stride = 2)
      dropout3 =  tf.contrib.layers.dropout(inputs=conv6_bn, keep_prob=0.5)
      
#       print("**************************************")
#       print(dropout3.shape)
      pool3_flat = tf.reshape(dropout3, [-1, 8* 8* 192])
    
      logits = tf.layers.dense(inputs=pool3_flat, units=10)
      predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.contrib.layers.softmax(logits)
  }
      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"]),
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [8]:
def current_loss():
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)


    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    print("######### Test accuracy #############\n",eval_result2)

In [10]:
# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_adam_SOTA_ADAM_nopool")


# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
print("started\n")
# Train the model
r = 140
for i in range (1,r):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=256,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps= 1000,
      hooks=None)
    # print("eva",type(eval_data))

    print ("\n\nEpoch = " )
    print(i)
    print("\n")
    current_loss()
print ("*****************END*******************")

started



Epoch = 
1


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.4724, 'loss': 1.8044226, 'global_step': 9001}
######### Test accuracy #############
 {'accuracy': 0.4672, 'loss': 1.8436011, 'global_step': 9001}


Epoch = 
2


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.49986, 'loss': 1.6356045, 'global_step': 10001}
######### Test accuracy #############
 {'accuracy': 0.4923, 'loss': 1.7033991, 'global_step': 10001}


Epoch = 
3


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.50206, 'loss': 1.6540952, 'global_step': 11001}
######### Test accuracy #############
 {'accuracy': 0.496, 'loss': 1.7666445, 'global_step': 11001}


Epoch = 
4


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.53508, 'loss': 1.4771334, 'global_step': 12001}
######### Test accuracy #############
 {'accuracy': 0.5239, 'loss': 1.5699438, 'global_step': 12001}


Epoch = 
5


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.23646, 'loss': 4.077588, 'global_step': 13001}
######### Test ac



Epoch = 
39


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.83854, 'loss': 0.45444718, 'global_step': 47001}
######### Test accuracy #############
 {'accuracy': 0.7277, 'loss': 0.94178456, 'global_step': 47001}


Epoch = 
40


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.84006, 'loss': 0.45989248, 'global_step': 48001}
######### Test accuracy #############
 {'accuracy': 0.7288, 'loss': 0.96192324, 'global_step': 48001}


Epoch = 
41


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.83986, 'loss': 0.46669233, 'global_step': 49001}
######### Test accuracy #############
 {'accuracy': 0.7301, 'loss': 0.9874788, 'global_step': 49001}


Epoch = 
42


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.84194, 'loss': 0.45591334, 'global_step': 50001}
######### Test accuracy #############
 {'accuracy': 0.7339, 'loss': 0.96503484, 'global_step': 50001}


Epoch = 
43


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.8462, 'loss': 0.44450086, 'global_step': 51001}
#########



Epoch = 
77


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.89958, 'loss': 0.3029303, 'global_step': 85001}
######### Test accuracy #############
 {'accuracy': 0.7337, 'loss': 1.2599722, 'global_step': 85001}


Epoch = 
78


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.90534, 'loss': 0.28162405, 'global_step': 86001}
######### Test accuracy #############
 {'accuracy': 0.7364, 'loss': 1.1955901, 'global_step': 86001}


Epoch = 
79


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.89368, 'loss': 0.3262754, 'global_step': 87001}
######### Test accuracy #############
 {'accuracy': 0.7273, 'loss': 1.2593712, 'global_step': 87001}


Epoch = 
80


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.9031, 'loss': 0.2913076, 'global_step': 88001}
######### Test accuracy #############
 {'accuracy': 0.7398, 'loss': 1.2301612, 'global_step': 88001}


Epoch = 
81


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.89348, 'loss': 0.31955767, 'global_step': 89001}
######### Test 

######### Test accuracy #############
 {'accuracy': 0.7455, 'loss': 1.3238498, 'global_step': 122001}


Epoch = 
115


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.91308, 'loss': 0.27475697, 'global_step': 123001}
######### Test accuracy #############
 {'accuracy': 0.731, 'loss': 1.4809705, 'global_step': 123001}


Epoch = 
116


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.92084, 'loss': 0.2580788, 'global_step': 124001}
######### Test accuracy #############
 {'accuracy': 0.736, 'loss': 1.4763297, 'global_step': 124001}


Epoch = 
117


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.9211, 'loss': 0.2436743, 'global_step': 125001}
######### Test accuracy #############
 {'accuracy': 0.7401, 'loss': 1.414727, 'global_step': 125001}


Epoch = 
118


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.92088, 'loss': 0.25070724, 'global_step': 126001}
######### Test accuracy #############
 {'accuracy': 0.7377, 'loss': 1.4261354, 'global_step': 126001}


Epoch = 
119


%%%

KeyboardInterrupt: 

In [ ]:
current_loss()